# One:
Register an app to get needed credentials ► https://developer.twitter.com/en/portal/petition/verification

Go to your App and generate an access token

## Hashtags or accounts related to potholes
"@mrpotholeuk",
"potholes",
"#PotholeDeaths",
"pothole damage".

## Hastags or accounts related to the solution
pothole detection

Related links https://www.teslarati.com/tesla-micro-maps-pothole-detection-update-elon-musk/

In [1]:
import tweepy
import json
import pandas as pd
from pandas.io.json import json_normalize

In [3]:
auth = tweepy.OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [4]:
api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [5]:
data = api.me()

In [ ]:
#My user
#print(json.dumps(data._json, indent=2))

In [6]:
mrpotholeuk_tweets = list(tweepy.Cursor(api.user_timeline, screen_name="mrpotholeuk", tweet_mode="extended").items(500))

In [8]:

# function to convert _json to JSON
def jsonify_tweepy(tweepy_object):
    json_str = json.dumps(tweepy_object._json)
    return json.loads(json_str)

# Call the function and unload each _json into follower_list
mrpotholeuk_df = json_normalize([jsonify_tweepy(tweet) for tweet in mrpotholeuk_tweets])
#mrpotholeuk_df = json_normalize([jsonify_tweepy(tweet.retweeted_status.full_text) if tweet.full_text.startswith("RT @") else jsonify_tweepy(tweet) for tweet in mrpotholeuk_tweets])

# Convert followers_list to a pandas dataframe
#mrpotholeuk_df = json_normalize(mrpotholeuk_list)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [10]:
mrpotholeuk_df.head(15)

,created_at,id,id_str,full_text,truncated,display_text_range,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,geo,coordinates,place,contributors,is_quote_status,retweet_count,favorite_count,favorited,retweeted,possibly_sensitive,lang,entities.hashtags,entities.symbols,entities.user_mentions,entities.urls,user.id,user.id_str,user.name,user.screen_name,user.location,user.description,user.url,user.entities.url.urls,user.entities.description.urls,user.protected,user.followers_count,user.friends_count,user.listed_count,...,retweeted_status.quoted_status.user.default_profile_image,retweeted_status.quoted_status.user.following,retweeted_status.quoted_status.user.follow_request_sent,retweeted_status.quoted_status.user.notifications,retweeted_status.quoted_status.user.translator_type,retweeted_status.quoted_status.geo,retweeted_status.quoted_status.coordinates,retweeted_status.quoted_status.place,retweeted_status.quoted_status.contributors,retweeted_status.quoted_status.is_quote_status,retweeted_status.quoted_status.retweet_count,retweeted_status.quoted_status.favorite_count,retweeted_status.quoted_status.favorited,retweeted_status.quoted_status.retweeted,retweeted_status.quoted_status.possibly_sensitive,retweeted_status.quoted_status.lang,retweeted_status.quoted_status.user.entities.url.urls,retweeted_status.quoted_status.user.profile_banner_url,retweeted_status.quoted_status.entities.media,retweeted_status.quoted_status.extended_entities.media,retweeted_status.place.id,retweeted_status.place.url,retweeted_status.place.place_type,retweeted_status.place.name,retweeted_status.place.full_name,retweeted_status.place.country_code,retweeted_status.place.country,retweeted_status.place.contained_within,retweeted_status.place.bounding_box.type,retweeted_status.place.bounding_box.coordinates,retweeted_status.quoted_status.quoted_status_id,retweeted_status.quoted_status.quoted_status_id_str,retweeted_status.quoted_status.quoted_status_permalink.url,retweeted_status.quoted_status.quoted_status_permalink.expanded,retweeted_status.quoted_status.quoted_status_permalink.display,quoted_status.quoted_status_id,quoted_status.quoted_status_id_str,quoted_status.quoted_status_permalink.url,quoted_status.quoted_status_permalink.expanded,quoted_status.quoted_status_permalink.display
0,Fri Feb 12 00:18:10 +0000 2021,1360020335672889344,1360020335672889344,How about using this money to fix #PotholeBrit...,False,"[0, 185]","<a href=""http://twitter.com/download/android"" ...",NaN,None,NaN,None,None,None,None,None,None,False,1,2,False,False,False,en,"[{'text': 'PotholeBritian', 'indices': [34, 49...",[],[],"[{'url': 'https://t.co/J0egqfmKGw', 'expanded_...",1667340889,1667340889,Mr Pothole,mrpotholeuk,,World's Best know Pothole Campaigner! Media go...,https://t.co/xwa7V30Dgc,"[{'url': 'https://t.co/xwa7V30Dgc', 'expanded_...","[{'url': 'https://t.co/PCTDjjicPl', 'expanded_...",False,5913,4812,67,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fri Feb 12 00:09:50 +0000 2021,1360018237904273414,1360018237904273414,@Tweet2Yorkshire https://t.co/iWF9kORkoH,False,"[17, 40]","<a href=""http://twitter.com/download/android"" ...",1.359982e+18,1359982299564171267,6.511581e+07,65115809,Tweet2Yorkshire,None,None,None,None,True,0,0,False,False,False,und,[],[],"[{'screen_name': 'Tweet2Yorkshire', 'name': 'T...","[{'url': 'https://t.co/iWF9kORkoH', 'expanded_...",1667340889,1667340889,Mr Pothole,mrpotholeuk,,World's Best know Pothole Campaigner! Media go...,https://t.co/xwa7V30Dgc,"[{'url': 'https://t.co/xwa7V30Dgc', 'expanded_...","[{'url': 'https://t.co/PCTDjjicPl', 'expanded_...",False,5913,4812,67,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fri Feb 12 00:09:38 +00

In [16]:
#mrpotholeuk_df.iloc[14][3]
mrpotholeuk_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Columns: 318 entries, created_at to quoted_status.quoted_status_permalink.display
dtypes: bool(18), float64(60), int64(9), object(231)
memory usage: 1.2+ MB


In [18]:
mrpotholeuk_words = list(tweepy.Cursor(api.search, q="mrpotholeuk", tweet_mode="extended").items(500))
mrpotholeuk_words_df = json_normalize([jsonify_tweepy(tweet) for tweet in mrpotholeuk_words])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [19]:
mrpotholeuk_words_df.head()

,created_at,id,id_str,full_text,truncated,display_text_range,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,geo,coordinates,contributors,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang,entities.hashtags,entities.symbols,entities.user_mentions,entities.urls,metadata.iso_language_code,metadata.result_type,user.id,user.id_str,user.name,user.screen_name,user.location,user.description,user.url,user.entities.url.urls,user.entities.description.urls,user.protected,user.followers_count,user.friends_count,user.listed_count,...,retweeted_status.quoted_status.user.notifications,retweeted_status.quoted_status.user.translator_type,retweeted_status.quoted_status.geo,retweeted_status.quoted_status.coordinates,retweeted_status.quoted_status.place,retweeted_status.quoted_status.contributors,retweeted_status.quoted_status.is_quote_status,retweeted_status.quoted_status.retweet_count,retweeted_status.quoted_status.favorite_count,retweeted_status.quoted_status.favorited,retweeted_status.quoted_status.retweeted,retweeted_status.quoted_status.possibly_sensitive,retweeted_status.quoted_status.lang,retweeted_status.possibly_sensitive,retweeted_status.entities.media,retweeted_status.extended_entities.media,entities.media,extended_entities.media,quoted_status.place.id,quoted_status.place.url,quoted_status.place.place_type,quoted_status.place.name,quoted_status.place.full_name,quoted_status.place.country_code,quoted_status.place.country,quoted_status.place.contained_within,quoted_status.place.bounding_box.type,quoted_status.place.bounding_box.coordinates,quoted_status.quoted_status_id,quoted_status.quoted_status_id_str,retweeted_status.place.id,retweeted_status.place.url,retweeted_status.place.place_type,retweeted_status.place.name,retweeted_status.place.full_name,retweeted_status.place.country_code,retweeted_status.place.country,retweeted_status.place.contained_within,retweeted_status.place.bounding_box.type,retweeted_status.place.bounding_box.coordinates
0,Fri Feb 12 01:19:25 +0000 2021,1360035750948069380,1360035750948069380,@PotholeOpinion @SimonWickers @mrpotholeuk @Do...,False,"[134, 400]","<a href=""http://twitter.com/download/iphone"" r...",1.360027e+18,1360026904221388802,1.774878e+08,177487798,PotholeOpinion,None,None,None,False,0,0,False,False,en,[],[],"[{'screen_name': 'PotholeOpinion', 'name': 'Ke...",[],en,recent,1177709329295822848,1177709329295822848,Smart Motorways Kill,MotorwaysKill,,Bringing a judicial review to force legal stop...,https://t.co/ma3jFWdtqv,"[{'url': 'https://t.co/ma3jFWdtqv', 'expanded_...","[{'url': 'https://t.co/OIz4tyVocA', 'expanded_...",False,734,401,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fri Feb 12 00:45:45 +0000 2021,1360027277946474496,1360027277946474496,@DominicBrowne1 @SimonWickers @mrpotholeuk @CP...,False,"[133, 179]","<a href=""http://twitter.com/download/iphone"" r...",1.359985e+18,1359984784706052098,8.550716e+17,855071556090941445,DominicBrowne1,None,None,None,False,0,0,False,False,en,[],[],"[{'screen_name': 'DominicBrowne1', 'name': 'Do...",[],en,recent,177487798,177487798,Keith Dixon,PotholeOpinion,UK,Retired from Road Surfacing. Cynical about Hig...,None,NaN,[],False,451,740,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fri Feb 12 00:44:39 +0000 2021,1360026998610010112,1360026998610010112,RT @DominicBrowne1: @SimonWickers @mrpotholeuk...,False,"[0, 140]","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,None,None,None,None,None,False,2,0,False,False,en,[],[],"[{'screen_name': 'DominicBrowne1', 'name': 'Do...",[],en,recent,177487798,177487798,Keith Dixon,PotholeOpinion,UK,Retired from Road Surfacing. Cynical about Hig...,None,NaN,[],False,451

In [41]:
display(mrpotholeuk_words_df.iloc[19][3])

'@Dorina335 @BBCNews @grantshapps @HighwaysEngland https://t.co/iWF9kORkoH'

## extracting also the retweets

In [49]:
def jsonify_tweepy_2(tweet_info):
    #json_str = json.dumps(tweepy_object._json)
    #return json.loads(json_str)

    if "retweeted_status" in dir(tweet_info):
      return tweet_info.retweeted_status.full_text
    else:
      return tweet_info.full_text

In [50]:
#mrpotholeuk_words = list(tweepy.Cursor(api.search, q="mrpotholeuk", tweet_mode="extended").items(500))
mrpotholeuk_words_df = pd.DataFrame([jsonify_tweepy_2(tweet) for tweet in mrpotholeuk_words])

In [97]:
mrpotholeuk_words_df.iloc[53][0]

'@mrpotholeuk @cpsuk @LilianGreenwood @CommonsTrans @grantshapps @MotorwaysKill @railandroad @CharlotteV I am a motoring lawyer and deal with all sorts of cases and also liaise with police all the time. Most police officers hate them too.'